# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import time
import warnings
warnings.filterwarnings('ignore')

# Load dataset

In [2]:
%time

IMG_SIZE = 224
BATCH_SIZE = 32
image_data_generator = ImageDataGenerator(rescale=1/255)

DATASET_DIR = "/content/drive/MyDrive/ML_PROJECTS/Computer_Vision/Facial_Emotion_Recognition/processed_data"

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.2 µs


## Spliting dataset into training, validation, and testing

In [3]:
%time

training_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split = 0.2,
    subset = 'training',
    seed = 42,
    image_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE
)

val_test_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset='validation',
    seed = 42,
    image_size =(IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE
)
val_size = int(0.5 * len(val_test_ds))
val_ds = val_test_ds.take(val_size)
test_ds = val_test_ds.skip(val_size)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs
Found 49807 files belonging to 7 classes.
Using 39846 files for training.
Found 49807 files belonging to 7 classes.
Using 9961 files for validation.


# Optimize dataset pipeline

In [4]:
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds):
    return ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y)).cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

train_ds = prepare(training_ds)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

# Data Augmentation

In [5]:
data_augmentation = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

# model

In [6]:
model = Sequential([
    data_augmentation,
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(training_ds.class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.build(input_shape=(None, IMG_SIZE, IMG_SIZE, 3))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,863 (42.61 MB)

 Trainable params: 11,169,863 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
%time
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    verbose=True
)
history

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.68 µs
Epoch 1/100


In [ ]:
def plot_accuracy(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

if 'history' in locals():
    plot_accuracy(history)
else:
    print("Model training history not found. Please train the model first.")

In [ ]:
def plot_loss(history):
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training loss')
    plt.plot(history.history['val_loss'], label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

if 'history' in locals():
    plot_accuracy(history)
else:
    print("Model training history not found. Please train the model first.")

In [ ]:
plot_loss(history)

# Predict Facial Emotion

In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_emotion_from_upload(model1, class_names, img_size):
    """
    Allows uploading an image, preprocesses it, and predicts the emotion using the model.

    Args:
        model: The trained Keras model.
        class_names: A list of emotion class names.
        img_size: The target image size (height, width) for the model.
    """
    uploaded = files.upload()

    for fn in uploaded.keys():
        img_path = fn
        img = image.load_img(img_path, target_size=(img_size, img_size))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)

        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions)
        predicted_emotion = class_names[predicted_class_index]

        print(f"Uploaded file: {fn}")
        print(f"Predicted Emotion: {predicted_emotion}")

if 'model' in locals() and 'training_ds' in locals():
    class_names = training_ds.class_names
    predict_emotion_from_upload(model, class_names, IMG_SIZE)
else:
    print("Model or class names not found. Please ensure the model is trained and class_names are available.")

# Save model

In [ ]:
model_save_path = '/content/drive/MyDrive/ML_PROJECTS/Computer_Vision/Facial_Emotion_Recognition/Model/modelV1.keras'
model.save(model_save_path)

print(f"Model saved successfully to {model_save_path}")

## Using transfer learning

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape = (IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = False
base_model.summary()

In [ ]:
model1 = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(len(training_ds.class_names), activation='softmax')
])

model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model1.build(input_shape=(None, IMG_SIZE, IMG_SIZE, 3))
model1.summary()

In [ ]:
history1 = model1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    verbose=True
)
history1

In [ ]:
plot_accuracy(hisory1)

In [ ]:
plot_loss(history1)

In [ ]:
model_save_path = '/content/drive/MyDrive/ML_PROJECTS/Computer_Vision/Facial_Emotion_Recognition/Model/modelV2.keras'
model.save(model_save_path)

print(f"Model saved successfully to {model_save_path}")